# Agrupamento e Redução de Dimensionalidade
## utilizando o algoritmo k-means
### Aluno: Rayana Souza Rocha
[Link do do Github da atividade de Rayana](https://github.com/rayanarocha/machinelearning/tree/master/agrupamento-e-redu%C3%A7%C3%A3o-de-dimensionalidade)

#### O que é K-Means?

K-Means é um algoritmo de clusterização (ou agrupamento) disponível na biblioteca Scikit-Learn.

É um algoritmo de aprendizado não supervisionado (ou seja, que não precisa de inputs de confirmação externos) que avalia e clusteriza os dados de acordo com suas características, como por exemplo:

lojas/centro logistico
clientes/produtos ou serviços semelhantes
clientes/características semelhantes
séries/gênero da série ou faixa etaria
usuarios de uma rede social/usuario influenciador
paciente/sintoma ou característica semelhante

Fonte: https://medium.com/programadores-ajudando-programadores/k-means-o-que-%C3%A9-como-funciona-aplica%C3%A7%C3%B5es-e-exemplo-em-python-6021df6e2572

### Como funciona?

![k-means.png](k-means.png)

1. Primeiro, preciso definir um ‘K’, ou seja, um número de clusters (ou agrupamentos).
2. Depois, preciso definir, aleatoriamente, um centroide para cada cluster.
3. O próximo passo é calcular, para cada ponto, o centroide de menor distância. Cada ponto pertencerá ao centroide mais próximo (lembrar do exemplo do CD logístico e das lojas: cada loja (ponto) deve ser atendida pelo CD (centróide) mais próximo)
4. Agora, devo reposicionar o centróide. A nova posição do centroide deve ser a média da posição de todos os pontos do cluster.
5. Os dois ultimos passos são repetidos, iterativamente, até obtermos a posição ideal dos centróides.

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
#Istalando `stopwords` em outros idiomas
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sklearn.cluster import MiniBatchKMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_sertanejo = pd.read_csv('letras-ptbr-sertanejo-grande.csv')
print('\nFormato do dataset test:',df_sertanejo.shape, '\n')
df_sertanejo.sample(3)


Formato do dataset test: (15381, 7) 



,SName,Lyric,Artist,Songs,Popularity,Genre,Genres
12860,A Fruta Vida,A fruta que eu mais gosto.. E que como todos o...,Teodoro e Sampaio,443,2.8,Sertanejo,Sertanejo; Forró; Country; Regional; Axé; Clás...
15064,Vem Cá Novinho,Oh pra fazer a brincadeira você vai ter que zo...,Mc Carol,21,10.7,Sertanejo,Funk Carioca; Pagode; Sertanejo; Axé; Rap; Funk
5630,O Melhor de Mim,E tudo que um dia eu quis foi te ver bem. Mas ...,Gusttavo Lima,266,31.1,Sertanejo,Sertanejo; Romântico; House; Funk Carioca; Axé...


In [3]:
df_sertanejo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15381 entries, 0 to 15380
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SName       15381 non-null  object 
 1   Lyric       15381 non-null  object 
 2   Artist      15381 non-null  object 
 3   Songs       15381 non-null  int64  
 4   Popularity  15381 non-null  float64
 5   Genre       15381 non-null  object 
 6   Genres      15381 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 841.3+ KB


#### Descrição dos dados:

- SName: Coluna que contém o título da música
- Lyric: Coluna que contém a letra da música
- Artist: Nome do artista ou banda
- Songs: Número de músicas do artista. Caso o mesmo artista tenha mais de uma música no conjunto, o valor dessa coluna estará repetido em todas as músicas.
- Popularity: Popularidade do artista. Caso o mesmo artista tenha mais de uma música no conjunto, o valor dessa coluna estará repetido em todas as músicas.
- Genre: Gênero da música.
- Genres: Gêneros possíveis.

#### Verificando o gênero musical dos dados. Todos são `sertanejo`

In [4]:
df_sertanejo['Genre'].value_counts()[0:10]

Sertanejo    15381
Name: Genre, dtype: int64

#### Verificando os artistas do dataset, é possível observar que o artista com mais músicas é a dupla `Teodoro e Sampaio`

In [5]:
df_sertanejo['Artist'].value_counts()[0:10]

Teodoro e Sampaio            432
Paula Fernandes              394
Tião Carreiro e Pardinho     378
Milionário e José Rico       370
Zezé Di Camargo e Luciano    368
Luan Santana                 332
Lourenço & Lourival          306
Chitãozinho e Xororó         299
Rick & Renner                297
Bruno e Marrone              275
Name: Artist, dtype: int64

#### Pré-processamento do texto

Procurar estilos de letra por conjuntos de músicas sertanejas que são semelhantes. Serão utilizadas as letras das músicas pra fazer o agrupamento

Removendo ruídos - `stopwords`

In [6]:
stop_words = set(stopwords.words("portuguese"))
print(len(stop_words))

207


In [7]:
#construindo uma nova lista para armazenar o texto limpo
clean_lyrics = []
for w in range(len(df_sertanejo.Lyric)):
    lyric = df_sertanejo['Lyric'][w]

    # removendo caracteres especiais e dígitos
    # removendo palavras com dois ou menos caracteres e alguns acordes contidos no dataset
    lyric = re.sub("(\\d|\\W)+|\w*\d\w*", " ", lyric)
    lyric = ' '. join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    clean_lyrics.append(lyric)

clean_lyrics[0:5]

['Não vou mais pensar você minha mente não vai entrar vim aqui avisar Teu psicológico preparar Que não vou mais esperar vou pegar todo mundo Virar vagabundo Depois que ficar com essa cidade inteira vai lembrar tanto que dei amor tanto que você não deu valor sua única chance vai ser alguma balada vida beijar sem perceber Sem ver que você avisei Tenta não vacilar Menina avisei Amor pra cuidar escapar mão cai chão Não tem conserto não Antes sumir sua vida mais uma coisa aqui portão última vez que vou falar Seu psicológico preparar Que não vou mais esperar Acabou minha paciência vou pegar todo mundo Virar vagabundo Depois que ficar com essa cidade inteira vai lembrar tanto que dei amor tanto que você não deu valor sua única chance Você não deu valor Você não deu valor Não deu valor não deu valor Amei demais agora não quero mais nem saber Deixa outras aproveitarem por você vou pegar todo mundo Virar vagabundo Depois que ficar com essa cidade inteira vai lembrar tanto que dei amor tanto que 

#### Vetores TF-IDF

Transformar o texto num vetor de características que serão utilizadas na comparação

In [8]:
#TF_IDF vetorização
tfv = TfidfVectorizer(
    min_df = 5,
    max_df = 0.9,
    max_features=None,
    stop_words=stop_words,
    ngram_range=(1,3)
)

#transformação
vec_text = tfv.fit_transform(clean_lyrics)

#retorna a lista de palavras
words = tfv.get_feature_names()

len(words)

c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


46139

In [9]:
words[1:10]

['aah',
 'aah aah',
 'aahh',
 'aai',
 'aba',
 'aba chapéu',
 'aba larga',
 'aba larga bruaca',
 'abafado']

# Agrupamento

#### Decidindo o K

In [10]:
#escolhendo k
#sdd - soma das distâncias quadráticas ao centro do grupo
qualidade = pd.DataFrame(columns=['k', 'ssd'])
for k in range(1,17, 1):
    kmeans = MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20)
    kmeans.fit(vec_text)
    qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)

alt.Chart(qualidade).mark_line(
    point=True
).encode(
    x = 'k',
    y = alt.Y('ssd', scale = alt.Scale(zero=False))
)

C:\Users\rayan\AppData\Local\Temp\ipykernel_13164\221130722.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\rayan\AppData\Local\Temp\ipykernel_13164\221130722.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\rayan\AppData\Local\Temp\ipykernel_13164\221130722.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\rayan\AppData\Local\Temp\ipykernel_13164\221130722.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

alt.Chart(...)

A partir do gráfico plotado anteriormente, é possível observar que entre os pontos 2 e 4 existe uma constância, por tanto, vou utilizar o valor 3 como quantidade dos centróides

In [11]:
kmeans = MiniBatchKMeans(n_clusters=3, init_size=1024, batch_size=2048, random_state=20)
#fit nos dados
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)

df_sertanejo_ag = df_sertanejo.assign(grupo = labels)

df_sertanejo_ag.sample(3)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres,grupo
4006,A Culpa É da Bebida (Part. Wesley Safadão),Mas um fim de noite. Nós na mesma cama. O que ...,Fred e Gustavo,106,1.2,Sertanejo,Romântico; Sertanejo; Country; Trilha Sonora; ...,0
11318,Foi Um Sonho,"Pra continuar, impossível. Quanta dor em mim. ...",Renan e Ray,41,1.0,Sertanejo,Sertanejo,1
11526,Feiticeira,Ela tem um jeito lindo de me olhar nos olhos. ...,Rick & Renner,343,5.0,Sertanejo,Sertanejo,2


#### Interpretando os grupos

In [12]:
df_sertanejo_ag['grupo'].value_counts()

1    7539
2    4019
0    3823
Name: grupo, dtype: int64

1. Utilize o k-means para procurar grupos de músicas, identificando o melhor número de grupos através das técnicas explicadas nas aulas. Em seguida, descreva os grupos encontrados, listando as palavras e os artistas mais frequentes em cada grupo. Tente nomear os grupos e explique em detalhes o racional para sua nomeação para cada grupo. Repita este processo utilizando como entrada para o k-means:
- O título da música;
- A letra da música;
- A concatenação do título e letra;

In [13]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-15:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '. join(words[word] for word in centroid))

0 : vai, pra, vou, quer, mulher, gente, pode, hoje, agora, tudo, amor, dia, sei, ver
1 : amor, pra, coração, vou, vida, mim, sei, tudo, saudade, gente, amar, tão, quero, paixão
2 : pra, vem, quero, deus, aqui, mim, vou, senhor, jesus, tudo, assim, ver, coração, amor


Como dito anteriormente, foram construídos 13 clusters que estão dividos em grupos de 0 a 12.

- 0 grupo: Malvada Pinga, fala da vontade da pessoa. do querer algo ou fazer algo com alguém.
- 1 grupo: Sertanejo Sofrência, pois fala muito em saudade, amor, coração, dor, tristeza, solidão
- 2 grupo: Sertanejo God, é um grupo de sertanejo católico, pois fala bastante em santos e santas e também utiliza bastante o termo **mariano**. termo muito utilizado no catolicismo em referência à Santa Maria, mãe de Deus, que fala de Deus e Jesus, mas no sentido de conexão Pai e filho, de cuidado, proteção com a vida, teraa, mundo, sertão.

In [14]:
pd.options.display.max_colwidth=100
df_sertanejo_ag.query('grupo == 2')[['SName', 'Lyric', 'Artist']].sample(5)

,SName,Lyric,Artist
1798,A Seca,"Quando o roceiro viu a terra esturricada. Na sua roça já cansada, seu olhar umedeceu. Com a falt...",Chitãozinho e Xororó
9022,Já Se Ouve,Já se ouve sobre a terra. O rumor qual de muito povo. É o clamor de reinos. E nações já congrega...,Ludmila Ferber
3101,O Show de Deus,Eu quero mostrar cantando o brilho da natureza. Eu vejo Deus trabalhando no seio dessa grandeza....,Daniel e Samuel
15279,Marcha do Remador,Se a canoa não virar. Olê olê olê olá. Eu chego lá. Se a canoa não virar. Olê olê olê olá. Eu ch...,Mc Marcelly
1995,Poema Sertanejo,Ah! se eu pudesse. Dedicaria ao meu sertão mais uma prece. Ah! mas não precisa não. Lá já existe...,Chitãozinho e Xororó


In [15]:
for g in range(0, 3):
    print('\n-----\nGrupo {}:'.format(g))
    print(df_sertanejo_ag.query('grupo == {}'. format(g))['Artist'].value_counts()[0:10])
    print('-------')


-----
Grupo 0:
Teodoro e Sampaio       179
Gabriel Diniz           128
Guilherme e Santiago     90
Gino & Geno              86
Gusttavo Lima            86
Fernando e Sorocaba      80
Rick & Renner            73
Henrique e Juliano       72
Luan Santana             72
Lucas Lucco              72
Name: Artist, dtype: int64
-------

-----
Grupo 1:
Milionário e José Rico       275
Zezé Di Camargo e Luciano    240
João Mineiro e Marciano      210
Paula Fernandes              200
Chitãozinho e Xororó         197
Roberta Miranda              187
Bruno e Marrone              180
Leonardo                     173
Matogrosso e Mathias         169
Eduardo Costa                169
Name: Artist, dtype: int64
-------

-----
Grupo 2:
Ludmila Ferber              212
Tião Carreiro e Pardinho    205
Daniel e Samuel             199
Paula Fernandes             156
Tonico e Tinoco             151
Suellen Lima                106
Sérgio Reis                 104
Luan Santana                 98
Lourenço & Louri

O grupo **Malvada Pinga** tem aqueles cantores ou duplas que cantam sertanejo mais contemporâneo, com mais adesão dos jovens. O grupo **Sertanejo Sofrência** tem as músicas para quem tá com o coração partido e também os cantores que tocam viola, que cantam a rotina da lida do dia a dia da vida no campo. O grupo **Sertanejo God** são os cantores sertanejos católicos que cantam a devoção do homem do campo a Deus e Nossa Senhora 

In [16]:
for g in range(0, 3):
    print('\n-----\nGrupo {}:'.format(g))
    print(df_sertanejo_ag.query('grupo == {}'. format(g))['SName'].value_counts()[0:10])
    print('-------')


-----
Grupo 0:
Vingança (part. MC Kekel)    4
Acordando o Prédio           4
Te Esperando                 4
Cantada                      4
A Paz Desse Amor             4
Chuva de Arroz               4
"A"                          4
Tô Mal                       3
Nova York                    3
Frente a Frente              3
Name: SName, dtype: int64
-------

-----
Grupo 1:
Perdoa                                5
Sofazinho (part. Jorge e Mateus)      4
Costumes                              4
Esperando Na Janela                   4
Te Vivo                               4
Pássaro de Fogo                       4
Tocando Em Frente (Part. Leonardo)    4
Meu Eu Em Você                        4
Não Precisa (Part. Victor e Leo)      4
Um Ser Amor                           4
Name: SName, dtype: int64
-------

-----
Grupo 2:
Nossa Senhora Aparecida                        5
Tudo Que Você Quiser                           4
Pra Você                                       4
Escreve Aí                 

In [17]:
for g in range(0, 3):
    print('\n-----\nGrupo {}:'.format(g))
    print(df_sertanejo_ag.query('grupo == {}'. format(g))['Lyric'].value_counts()[0:10])
    print('-------')


-----
Grupo 0:
Uh, uh, uh, uh. Não vou mais pensar em você. Na minha mente cê não vai entrar. E só vim aqui te avisar. Teu psicológico preparar. Que eu não vou mais te esperar. Eu vou pegar todo mundo. Virar um vagabundo. Depois que eu ficar com essa cidade inteira. Aí cê vai lembrar. Do tanto que eu te dei amor. E o tanto que você não deu valor. E a sua única chance vai ser. Em alguma balada da vida. Eu te beijar sem perceber. Sem ver que é você. Uh, uh, uh, uh. Eu te avisei. Tenta não vacilar. Menina, eu te avisei. Amor é pra cuidar. Se escapar da mão, cai no chão. Não tem conserto, não. Antes de sumir da sua vida. Só mais uma coisa. Tô aqui no portão. E é a última vez que eu vou falar. Seu psicológico preparar. Que eu não vou mais te esperar. Acabou minha paciência. Eu vou pegar todo mundo. Virar um vagabundo. Depois que eu ficar com essa cidade inteira. Aí cê vai lembrar. Do tanto que eu te dei amor. E o tanto que você não deu valor. É sua única chance. Você não deu valor. Você nã

In [18]:
df_sertanejo_ag.query('Artist == "Bruno e Marrone"')[['grupo', 'SName']].groupby('grupo').count()

,SName
grupo,
0,62
1,180
2,33


#### Visualizando os grupos em muitas dimensões

In [19]:
df_sertanejo_embedded = TSNE(n_components=2, verbose=1, perplexity=45).fit_transform(vec_text)

[t-SNE] Computing 136 nearest neighbors...
[t-SNE] Indexed 15381 samples in 0.005s...


c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computed neighbors for 15381 samples in 11.428s...
[t-SNE] Computed conditional probabilities for sample 1000 / 15381
[t-SNE] Computed conditional probabilities for sample 2000 / 15381
[t-SNE] Computed conditional probabilities for sample 3000 / 15381
[t-SNE] Computed conditional probabilities for sample 4000 / 15381
[t-SNE] Computed conditional probabilities for sample 5000 / 15381
[t-SNE] Computed conditional probabilities for sample 6000 / 15381
[t-SNE] Computed conditional probabilities for sample 7000 / 15381
[t-SNE] Computed conditional probabilities for sample 8000 / 15381
[t-SNE] Computed conditional probabilities for sample 9000 / 15381
[t-SNE] Computed conditional probabilities for sample 10000 / 15381
[t-SNE] Computed conditional probabilities for sample 11000 / 15381
[t-SNE] Computed conditional probabilities for sample 12000 / 15381
[t-SNE] Computed conditional probabilities for sample 13000 / 15381
[t-SNE] Computed conditional probabilities for sample 14000 / 1538

2. Mesmo que 1 acima, mas aplique uma redução de dimensionalidade utilizando o PCA, e utilize os dados transformados como entrada para o k-means.

#### PCA - Principal Component Analysis

![k-means.png](pca.png)

Redução de Dimensionalidade

A Análise de Componentes Principais ou PCA (Principal Component Analysis) é uma técnica de análise multivariada que pode ser usada para analisar inter-relações entre um grande número de variáveis e explicar essas variáveis em termos de suas dimensões inerentes (Componentes).

O objetivo é encontrar um meio de condensar a informação contida em várias variáveis originais em um conjunto menor de variáveis estatísticas (componentes) com uma perda mínima de informação.

O número de componentes principais se torna o número de variáveis consideradas na análise, mas geralmente as primeiras componentes são as mais importantes já que explicam a maior parte da variação total.

As componentes principais em geral são extraídas via matriz de covariância, mas também podem ser extraídas via matriz de correlação.

Fonte: https://site.statplace.com.br/blog/analise-de-componentes-principais/

In [25]:
svd = TruncatedSVD(n_components=2).fit_transform(vec_text)
print(svd)

[[ 0.15039815 -0.09084745]
 [ 0.08097123  0.02701306]
 [ 0.12439639 -0.08794412]
 ...
 [ 0.11749802 -0.10551826]
 [ 0.07921003 -0.06663635]
 [ 0.07595487 -0.04887906]]


3. Compare a qualidade dos grupos com e sem PCA usando o coeficiente de silhueta.

### Silhouette Clustering

In [21]:
range_n_clusters = [3, 6, 9, 11, 13]
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(svd)
    centers = clusterer.cluster_centers_

    score = silhouette_score(svd, preds)
    print("Para n_clusters = {}, a pontuação da silhoueta é {})".format(n_clusters, score))

Para n_clusters = 3, a pontuação da silhoueta é 0.36645847836672574)
Para n_clusters = 6, a pontuação da silhoueta é 0.33320331242383133)
Para n_clusters = 9, a pontuação da silhoueta é 0.31597041249427754)
Para n_clusters = 11, a pontuação da silhoueta é 0.32279955361456164)
Para n_clusters = 13, a pontuação da silhoueta é 0.3202672431718631)


4. Utilize o PCA e t-SNE e gere visualizações dos grupos encontrados nas questões anteriores. As visualizações indicam que os grupos são bem separados? Por que você acha que sim ou que não?

# t-sne

In [22]:
df_sertanejo_ag = df_sertanejo_ag.assign(tsne1 = df_sertanejo_embedded[:,0], tsne2 = df_sertanejo_embedded[:,1])

alt.Chart(df_sertanejo_ag.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2',
    color = 'grupo:N',
    tooltip=['Artist', "SName"]
).interactive()

c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [23]:
centers = kmeans.cluster_centers_
print(centers)

[[1.23018770e-04 3.51237274e-04 1.18614256e-04 ... 4.46781342e-04
  3.20889489e-05 2.53706825e-04]
 [7.98033212e-05 2.12660974e-04 1.24571359e-04 ... 4.46701048e-04
  2.23719366e-04 1.74913981e-04]
 [0.00000000e+00 1.86510512e-04 8.78369938e-05 ... 1.02386004e-04
  2.58129555e-05 7.03636908e-04]]


# svd

Utilizei o `TruncatedSVD`, pois, obtive um erro, e segundo as pesquisas o PCA obtém esse erro quando tenta manipular matriz esparsa.

In [24]:
df_sertanejo_ag = df_sertanejo_ag.assign(pca1 = svd[:,0], pca2 = svd[:,1])

alt.Chart(df_sertanejo_ag.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'pca1',
    y = 'pca2',
    color = 'grupo:N',
    tooltip=['Artist', "SName"]
).interactive()

c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

Os clusters não estão bem definidos no **t-sne**, porém no **svd** é possível observar com mais clareza o agrupamento. No gráfico do `t-sne`, tanto os dados não estão bem agrupados, como também estão bem afastados uns dos outros, talvez isso aconteça pela repetiçao de palavras iguais ou semelhantes entre os grupos, por muito artistas estarem em muitos grupos distintos ao mesmo tempo.

Em termos de visualização o `SVD` se saiu melhor que o `t-SNE`.